## Разметка датасета

In [ ]:
import numpy as np
import pandas as pd


df = pd.read_csv("tg_messages_mriya.csv", encoding="utf-8")
df

,name,additional_id,date,rating,text,answer
0,5.198251e+09,1.328529e+09,1.744522e+09,0,Ну Мрия это ещё не весь Крым☺️,NaN
1,1.759388e+09,1.328529e+09,1.744467e+09,1,МРИЯ - это КОСМОС❤️,NaN
2,1.749466e+09,1.328529e+09,1.744464e+09,1,"__🧑‍🚀 Поехали!__\n\nНе только в космос, но и в...",NaN
3,1.749466e+09,1.328529e+09,1.744445e+09,1,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,NaN
4,1.749466e+09,NaN,1.744445e+09,1,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,NaN
...,...,...,...,...,...,...
329,1.749466e+09,1.328529e+09,1.728660e+09,0,"Мы уверены, что форум [Новые горизонты](https:...",NaN
330,1.749466e+09,NaN,1.728660e+09,0,"Мы уверены, что форум [Новые горизонты](https:...",NaN
331,1.749466e+09,1.328529e+09,1.728654e+09,0,Фрагмент выступления генерального директора ку...,NaN
332,1.749466e+09,NaN,1.728654e+09,0,Фрагмент выступления генерального директора ку...,NaN


In [ ]:
labels = df['rating'].to_list()
for i, line in df.iterrows():
    if labels[i].is_integer() or line['text'] is np.nan:
        continue

    text = line['text']
    print(i)
    if len(text) > 100:
        curlen = 0
        for subtext in text.split(' '):
            print(subtext, end=' ')
            curlen += len(subtext)
            if curlen > 100:
                curlen = 0
                print()
    else:
        print(text)

    print('\n')

    try:
        label = None
        while label is None:
            try:
                label = int(input("Категория (0 - neutral, 1 - positive, 2 - negative):\n"))
            except ValueError:
                label = None
    except:
        break

    labels[i] = label
    print()

df['rating'] = labels

0
Ну Мрия это ещё не весь Крым☺️




In [ ]:
df.loc[~df['rating'].isna(), ['text', 'rating']]

,text,rating
0,Ну Мрия это ещё не весь Крым☺️,1.0
1,МРИЯ - это КОСМОС❤️,2.0
2,"__🧑‍🚀 Поехали!__\n\nНе только в космос, но и в...",2.0
3,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,2.0
4,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,2.0
...,...,...
329,"Мы уверены, что форум [Новые горизонты](https:...",1.0
330,"Мы уверены, что форум [Новые горизонты](https:...",1.0
331,Фрагмент выступления генерального директора ку...,1.0
332,Фрагмент выступления генерального директора ку...,1.0


In [ ]:
df.dropna(axis=0, how='all').to_csv("tg_messages_mriya.csv", index=False)

## Предподготовка данных

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd


max_samples = 60000  # максимальное количество элементов в датасете

def sample_dataset(df, sample_size):
    proportions = df['label'].value_counts(normalize=True)
    return (df.groupby('label')
            .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))
            .reset_index(drop=True))


test_datasets = {}
df = pd.read_csv("drive/MyDrive/Cource work/tg_messages_mriya.csv")
test_datasets['tg_messages_mriya'] = df[['text', 'rating']].rename(columns={'rating': 'label'})
test_datasets['tg_messages_mriya'].head()

,text,label
0,Ну Мрия это ещё не весь Крым☺️,0
1,МРИЯ - это КОСМОС❤️,1
2,"__🧑‍🚀 Поехали!__\n\nНе только в космос, но и в...",1
3,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,1
4,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,1


### Sentiment of bank reviews

In [3]:
df = pd.read_csv("drive/MyDrive/Cource work/sentiment_of_bank_reviews.csv", sep='\t')
df.head()

,idx,Score,Text
0,0,Positive,В Альфа-Банке работает замечательная девушка -...
1,1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,Positive,Очень порадовала оперативность работы в банке....
3,3,Negative,Имела неосторожность оформить потреб. кредит в...
4,4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...


In [4]:
df['Score'].value_counts()

,count
Score,
Positive,7000
Negative,6999


In [5]:
df = df[['Text', 'Score']].rename(columns={'Text': 'text', 'Score': 'label'})
label_repl = {'Negative': 2, 'Positive': 1}
df['label'] = df['label'].apply(lambda x: label_repl[x])

test_datasets['sentiment_of_bank_reviews'] = df
test_datasets['sentiment_of_bank_reviews'].head()

,text,label
0,В Альфа-Банке работает замечательная девушка -...,1
1,Оформляя рассрочку в м. Видео в меге тёплый ст...,2
2,Очень порадовала оперативность работы в банке....,1
3,Имела неосторожность оформить потреб. кредит в...,2
4,Небольшая предыстория: Нашел на сайте MDM банк...,2


### RuTweetCorp

In [6]:
df_pos = pd.read_csv('drive/MyDrive/Cource work/RuTweetCorp_positive.csv')
df_neg = pd.read_csv('drive/MyDrive/Cource work/RuTweetCorp_negative.csv')
df_pos.head()

,id,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [7]:
df_pos = pd.concat([df_pos[['ttext']], pd.DataFrame({'label': [1] * len(df_pos)})], axis=1)
df_neg = pd.concat([df_neg[['ttext']], pd.DataFrame({'label': [2] * len(df_neg)})], axis=1)
df = pd.concat([df_pos, df_neg], axis=0).rename(columns={'ttext': 'text'})
df = df.sample(frac=1).reset_index(drop=True)

test_datasets['rutweetcorp'] = df
test_datasets['rutweetcorp'].head()

,text,label
0,"фак, завтра в парикмахерскую к новому парикмах...",2
1,Танцевать сальсу без партнера даже весело)\nБл...,1
2,"вот так и помогай друзьям, когда они просят......",2
3,"""Без меня ты НИКТО!"" - сказал интернет компьют...",1
4,"RT @Pavsekakii: На улице запах ""Нового года"", ...",1


In [8]:
test_datasets['rutweetcorp']['label'].value_counts()

,count
label,
1,114911
2,111923


In [9]:
test_datasets['rutweetcorp'] = sample_dataset(test_datasets['rutweetcorp'], max_samples)
test_datasets['rutweetcorp']['label'].value_counts()

<ipython-input-2-67c635bd206d>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))


,count
label,
1,30395
2,29605


### RuReviews

In [10]:
df = pd.read_csv('drive/MyDrive/Cource work/rureviews.csv', sep='\t')
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [11]:
df['sentiment'].value_counts()

,count
sentiment,
negative,30000
neautral,30000
positive,30000


In [12]:
df = df.rename(columns={'review': 'text', 'sentiment': 'label'})
label_repl = {'negative': 2, 'neautral': 0, 'positive': 1}
df['label'] = df['label'].apply(lambda x: label_repl[x])

test_datasets['rureviews'] = df
test_datasets['rureviews'].head()

,text,label
0,качество плохое пошив ужасный (горловина напер...,2
1,"Товар отдали другому человеку, я не получила п...",2
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",2
3,"товар не пришел, продавец продлил защиту без м...",2
4,"Кофточка голая синтетика, носить не возможно.",2


In [13]:
test_datasets['rureviews'] = sample_dataset(test_datasets['rureviews'], max_samples)
test_datasets['rureviews']['label'].value_counts()

<ipython-input-2-67c635bd206d>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))


,count
label,
0,20000
1,20000
2,20000


### RuSentiment

In [14]:
df_preselected = pd.read_csv('drive/MyDrive/Cource work/rusentiment_preselected_posts.csv')
df_random = pd.read_csv('drive/MyDrive/Cource work/rusentiment_random_posts.csv')
df_test = pd.read_csv('drive/MyDrive/Cource work/rusentiment_test.csv')
df = pd.concat([df_preselected, df_random, df_test], axis=0)[['text', 'label']]
df.head()

,text,label
0,Прорвём информационную блокаду изнутри.,neutral
1,"Никогда у меня не будет ""одного приложения для...",negative
2,"Кури-и тебя не укусит злая собака, потому что ...",skip
3,"Есть 3 типа людей:\nУмные, которые делают все ...",neutral
4,мегафон чет накрыло,neutral


In [15]:
df['label'].value_counts()

,count
label,
neutral,12720
positive,6646
skip,4440
negative,3912
speech,3467


In [16]:
label_repl = {'neutral': 0, 'positive': 1, 'skip': 0, 'negative': 2, 'speech': 0}
df['label'] = df['label'].apply(lambda x: label_repl[x])

test_datasets['rusentiment'] = df
test_datasets['rusentiment'] .head()

,text,label
0,Прорвём информационную блокаду изнутри.,0
1,"Никогда у меня не будет ""одного приложения для...",2
2,"Кури-и тебя не укусит злая собака, потому что ...",0
3,"Есть 3 типа людей:\nУмные, которые делают все ...",0
4,мегафон чет накрыло,0


### ru_sentiment_MonoHime

In [17]:
df = pd.read_csv('drive/MyDrive/Cource work/ru_sentiment_monohime.csv', index_col=0)
df.head()

,text,sentiment
43956,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1
17755,Отель выбрали потому что рядом со стадионом. О...,0
20269,"Вылечили\nГноился с рождения глазик, в поликли...",1
16648,Хорошее расположение.С вокзала дошли пешком.Но...,0
27879,"Отличное месторасположение,прекрасный вид,особ...",1


In [18]:
df['sentiment'].value_counts()

,count
sentiment,
1,100792
2,55310
0,54887


In [19]:
df = df.rename(columns={'sentiment': 'label'}).reset_index(drop=True)
test_datasets['ru_sentiment_monohime'] = df
test_datasets['ru_sentiment_monohime'].head()

,text,label
0,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1
1,Отель выбрали потому что рядом со стадионом. О...,0
2,"Вылечили\nГноился с рождения глазик, в поликли...",1
3,Хорошее расположение.С вокзала дошли пешком.Но...,0
4,"Отличное месторасположение,прекрасный вид,особ...",1


In [20]:
test_datasets['ru_sentiment_monohime'] = sample_dataset(test_datasets['ru_sentiment_monohime'], max_samples)
test_datasets['ru_sentiment_monohime']['label'].value_counts()

<ipython-input-2-67c635bd206d>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))


,count
label,
1,28663
2,15729
0,15608


In [21]:
test_datasets

{'tg_messages_mriya':                                                   text  label
 0                       Ну Мрия это ещё не весь Крым☺️      0
 1                                  МРИЯ - это КОСМОС❤️      1
 2    __🧑‍🚀 Поехали!__\n\nНе только в космос, но и в...      1
 3    **Мечты о звёздах начинаются с Земли!**\n\nИ е...      1
 4    **Мечты о звёздах начинаются с Земли!**\n\nИ е...      1
 ..                                                 ...    ...
 329  Мы уверены, что форум [Новые горизонты](https:...      0
 330  Мы уверены, что форум [Новые горизонты](https:...      0
 331  Фрагмент выступления генерального директора ку...      0
 332  Фрагмент выступления генерального директора ку...      0
 333  __В форуме __[__Новые горизонты__](https://t.m...      0
 
 [334 rows x 2 columns],
 'sentiment_of_bank_reviews':                                                     text  label
 0      В Альфа-Банке работает замечательная девушка -...      1
 1      Оформляя рассрочку в м. Видео

## Тестирование

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification; tokenizer = AutoTokenizer.from_pretrained("SCANSKY/distilbertTourism-multilingual-sentiment"); model = AutoModelForSequenceClassification.from_pretrained("SCANSKY/distilbertTourism-multilingual-sentiment")

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)

MODEL_NAMES = [
    # "blanchefort/rubert-base-cased-sentiment",
    # "seara/rubert-tiny2-russian-sentiment",
    # "cointegrated/rubert-tiny-sentiment-balanced",
    "MonoHime/rubert_conversational_cased_sentiment",
    "sismetanin/rubert-ru-sentiment-rureviews",
    "blanchefort/rubert-base-cased-sentiment-rurewiews",
    "rajora/distilbert-multilingual-sentiment"
]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 12
MAX_LENGTH = 512

In [23]:
import gc
from tqdm import tqdm


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

def evaluate_model(model, dataloader, model_name, label_consistance=True):
    model.eval()
    predictions, true_labels = [], []

    i = 1
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating {model_name}"):
            try:
                inputs = {
                    "input_ids": batch["input_ids"].to(DEVICE),
                    "attention_mask": batch["attention_mask"].to(DEVICE)
                }
                outputs = model(**inputs)
                logits = outputs.logits
                if not label_consistance:
                    logits = outputs.logits[:, 1:]

                predictions.extend(torch.argmax(logits, dim=1).cpu().tolist())
                true_labels.extend(batch["labels"].cpu().tolist())

                if i % 25 == 0:
                    del inputs
                    del outputs
                    gc.collect()

                i += 1
            finally:
                continue

    return predictions, true_labels

In [ ]:
results = []
for model_name in MODEL_NAMES:
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name).to(DEVICE)
    except Exception as e:
        print(f"Ошибка загрузки модели {model_name}: {str(e)}")
        raise e

    for dataset_name, df in list(test_datasets.items()):
        try:
            texts = df["text"].tolist()
            labels = df["label"].tolist()

            # Автоматическое определение количества классов
            labels_are_consistent = model.config.num_labels == len(set(labels))
            if model.config.num_labels == 2:
                labels = [label if label == 0 else label - 1 for label in labels]
                labels_are_consistent = True
            elif not labels_are_consistent:
                print(f"Модель {model_name} (классы={model.config.num_labels}) не совпадает с {dataset_name}")
                labels = [label - 1 for label in labels]

            dataset = CustomDataset(texts, labels, tokenizer)
            dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, pin_memory=True)

            preds, truth = evaluate_model(model, dataloader, model_name, labels_are_consistent)

            metrics = {
                "model": model_name,
                "dataset": dataset_name,
                "accuracy": accuracy_score(truth, preds),
                "f1_weighted": f1_score(truth, preds, average="weighted", zero_division=0.),
                "f1_macro": f1_score(truth, preds, average="macro", zero_division=0.),
                "classification_report": classification_report(truth, preds, zero_division=0.),
                "confusion_matrix": confusion_matrix(truth, preds).tolist()
            }
            print(metrics)
            results.append(metrics)
        finally:
            results_df = pd.DataFrame(results)
            results_df.to_csv("drive/MyDrive/Cource work/model_evaluation_results3.csv", index=False)
            results_df.to_excel("drive/MyDrive/Cource work/model_evaluation_results3.xlsx", index=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at MonoHime/rubert_conversational_cased_sentiment and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating MonoHime/rubert_conversational_cased_sentiment: 100%|██████████| 28/28 [00:09<00:00,  2.82it/s]


{'model': 'MonoHime/rubert_conversational_cased_sentiment', 'dataset': 'tg_messages_mriya', 'accuracy': 0.8982035928143712, 'f1_weighted': 0.9275362776556592, 'f1_macro': 0.5008790436005626, 'classification_report': '              precision    recall  f1-score   support\n\n           0       0.98      0.91      0.95       327\n           1       0.03      0.14      0.06         7\n\n    accuracy                           0.90       334\n   macro avg       0.51      0.53      0.50       334\nweighted avg       0.96      0.90      0.93       334\n', 'confusion_matrix': [[299, 28], [6, 1]]}


Evaluating MonoHime/rubert_conversational_cased_sentiment:  79%|███████▉  | 927/1167 [06:01<01:39,  2.41it/s]